In [150]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import pandas as pd
import re

In [151]:
url = 'https://www.dorts.gov.taipei/News_toggle.aspx?n=2A66A485FACB0D5B&sms=87415A8B9CE81B16&PageSize=400'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
df_result= pd.DataFrame(columns=['Time', 'Url', 'Question', 'Answer', '點閱數', '資料更新', '資料檢視', '資料維護'])

In [164]:
question_list = []
answer_list = []
current_time_list = []

x = 177
for index in range(1, x + 1):
    QA_items = soup.find_all('li', {'data-index': str(index)})
    

    for QA_item in QA_items:
        questions = QA_item.find_all('div', class_='div')
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        answers = QA_item.find_all('div', class_='accordion-body__contents')
        
        for question in questions:
            question_list.append(question['title'])
            current_time_list.append(current_time)
        
        for answer in answers:
            answer_list.append(answer.text.strip())
    
    data_update_list = []
    data_update = soup.find_all('div', class_='accordion-footer')
    for li in data_update:
        data_update_value = li.find('li')
        if data_update_value and '資料更新' in data_update_value.text:
            update_info = data_update_value.text.strip().replace('資料更新：', '')
            data_update_list.append(update_info)

    data_review_list = []     
    pattern = re.compile(r'資料檢視：')
    data_review_elements = soup.find_all('li', text=pattern)
    for data_review in data_review_elements:
        data_review_value = data_review.text.strip().replace('資料檢視：', '')
        data_review_list.append(data_review_value)


    data_maintenance_list = []      
    pattern = re.compile(r'資料維護：')
    data_maintenance_elements = soup.find_all('li', text=pattern)
    for data_maintenance in data_maintenance_elements:
        data_maintenance_value = data_maintenance.text.strip().replace('資料維護：', '')
        data_maintenance_list.append(data_maintenance_value)
        
df_result['Question'] = question_list
df_result['Answer'] = answer_list
df_result['Time'] = current_time_list
df_result['資料更新'] = data_update_list
df_result['資料檢視'] = data_review_list
df_result['資料維護'] = data_maintenance_list           

/var/folders/7t/x585800j10v6hqth9nt23qp80000gn/T/ipykernel_50899/1773161239.py:34: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  data_review_elements = soup.find_all('li', text=pattern)
/var/folders/7t/x585800j10v6hqth9nt23qp80000gn/T/ipykernel_50899/1773161239.py:42: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  data_maintenance_elements = soup.find_all('li', text=pattern)


In [163]:
df_result.to_csv('捷運工程局QA.csv', index=False)